# フレア検出 学習ノート 01: データ読み込みと前処理

このノートでは、TESS の light curve FITS ファイルから
`BaseFlareDetector` が扱う "正規化済み光度配列" を作るまでの流れを、
実際のコードと Python 文法のポイントを交えながら学びます。

先に `flare_learning_overview.ipynb` を一通り実行しておくと、
ここで出てくるクラスやディレクトリ構成がイメージしやすくなります。


## このノートでできるようになること

- `Path` / `PROJECT_ROOT` を使って、リポジトリ内のファイルに安全にアクセスできる。
- TESS LC FITS ファイルの「どこに」「どのようなカラム」があるかを確認できる。
- `BaseFlareDetector.load_TESS_data()` が
  - ファイル名からデータ名・セクタ番号を取り出し
  - 適切なフラックス列（`PDCSAP_FLUX` / `SAP_FLUX`）を選び
  - NaN を取り除き、平均フラックスで正規化
    していることを理解できる。
- NumPy のブールマスク（`~np.isnan(...)` など）の基本を具体例で説明できる。


## 1. セットアップ: Path とプロジェクトルート

まずは、このノートブックがどこに置かれていて、
プロジェクト全体のルートがどこかを `pathlib.Path` で取得します。

ここでは以下の Python 文法にも注目します。

- `from pathlib import Path` でクラス `Path` をインポート
- `Path().resolve()` で「このノートブックが動いているカレントディレクトリ」を絶対パスで取得
- `NOTEBOOK_DIR.parent` で 1 つ上のディレクトリ（= プロジェクトルート）を求める
- `if str(PROJECT_ROOT) not in sys.path:` で import パスに安全に追加する


In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

### コードの解説（Python 文法のポイント）

- `from pathlib import Path`
  - `pathlib` は、OS に依存しないパス操作のための標準ライブラリです。
  - `Path("foo") / "bar"` のように `/` 演算子でパスを結合できます（文字列連結ではなく、パス結合）。
- `NOTEBOOK_DIR = Path().resolve()`
  - 引数なしの `Path()` は「カレントディレクトリ」を表します。
  - `.resolve()` でシンボリックリンクなどを解決した絶対パスを取得します。
- `PROJECT_ROOT = NOTEBOOK_DIR.parent`
  - `Path` オブジェクトの `.parent` は 1 つ上のディレクトリを表します。
  - このリポジトリでは、`notebooks/` の 1 つ上がプロジェクトルートになっています。
- `sys.path.insert(0, str(PROJECT_ROOT))`
  - `sys.path` は Python がモジュールを探すパスのリストです。
  - 先頭にプロジェクトルートを追加することで、`src.flarepy_EK_Dra` などを直接 import できます。


## 2. TESS FITS ファイルの場所を確認する

次に、EK Dra の TESS light curve FITS ファイルが
どこに置かれているかを `Path` で確認します。

ここでは以下の点に注目します。

- パスの連結: `PROJECT_ROOT / "data" / "TESS" / "EK_Dra"`
- ディレクトリの存在確認: `DATA_DIR.exists()`
- `glob("*.fits")` によるファイル一覧の取得
- `sorted(...)` とスライス `[:5]` による「はじめの数件だけ」の確認


In [ ]:
DATA_DIR = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
print("DATA_DIR:", DATA_DIR)
print("DATA_DIR.exists():", DATA_DIR.exists())

fits_files = []
if not DATA_DIR.exists():
    print("Warning: データディレクトリが見つかりません。パスや data/ の配置を確認してください。")
else:
    fits_files = sorted(DATA_DIR.glob("*.fits"))
    print("found", len(fits_files), "FITS files")
    print("first 5 files:")
    for path in fits_files[:5]:
        print("  -", path.name)

fits_files[:3]  # Jupyter 上で中身をざっくり確認するための行（Path オブジェクトのリスト）

### コードの解説（Path / glob / スライス）

- `DATA_DIR = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"`
  - `/` 演算子でパスを順に結合しています。
  - 文字列連結（`+`）と違い、OS ごとの区切り文字を自動で処理してくれます。
- `DATA_DIR.exists()`
  - 戻り値は `True` / `False` のブール値です。
  - 存在しないディレクトリに対して後続の処理を走らせないためのガードとして使います。
- `DATA_DIR.glob("*.fits")`
  - `*.fits` というパターンにマッチするファイルを `Path` のイテレータとして返します。
  - `sorted(...)` で時系列順（だいたいファイル名順）に並べ替えています。
- `fits_files[:5]`
  - リストの先頭 5 件だけを取り出すスライス構文です。
  - `[:3]` なら先頭 3 件、`[2:5]` なら 2〜4 番目（0 始まり）を意味します。


## 3. FITS ファイルの中身を少し覗いてみる

実際にどんなカラムが入っているかを、`astropy.io.fits` を使って確認します。

ここでは以下の点に注目します。

- `with fits.open(path) as hdul:` というコンテキストマネージャ構文
- HDU (Header Data Unit) の考え方
- `hdul[1].columns` で 1 番目の拡張に含まれるカラム一覧を表示する方法


In [ ]:
import astropy.io.fits as fits

if not DATA_DIR.exists() or not fits_files:
    print("DATA_DIR もしくは FITS ファイルが見つからないため、このセルはスキップします。")
else:
    example_path = fits_files[0]
    print("example_path:", example_path)

    # with 構文: ブロックを抜けると自動でファイルを閉じてくれる
    with fits.open(example_path) as hdul:
        print("HDU list length:", len(hdul))
        # 通常、light curve データは拡張HDU (index 1) に入っている
        data_hdu = hdul[1]
        print("Columns in HDU[1]:")
        print(data_hdu.columns)

        # TIME や PDCSAP_FLUX の一部だけ中身を見てみる
        time_sample = data_hdu.data["TIME"][:5]
        flux_sample = data_hdu.data["PDCSAP_FLUX"][:5]
        print("TIME sample:", time_sample)
        print("PDCSAP_FLUX sample:", flux_sample)

### コードの解説（with 構文 / HDU / カラム）

- `with fits.open(example_path) as hdul:`
  - `with` はコンテキストマネージャ構文と呼ばれ、ブロックを抜けたときに自動的に後片付け（ここではファイルクローズ）をしてくれます。
  - `hdul` は "HDU list" の略で、`HDU` オブジェクトのリストのようなものです。
- `hdul[0]` / `hdul[1]`
  - `hdul[0]`: Primary HDU（主にヘッダー）
  - `hdul[1]`: 実際の light curve データが入っている拡張 HDU
- `data_hdu.columns`
  - 利用可能なカラム名と型の一覧が表示されます。
  - 典型的には `TIME`, `PDCSAP_FLUX`, `PDCSAP_FLUX_ERR`, `SAP_FLUX`, `SAP_FLUX_ERR` などが含まれます。
- `data_hdu.data["TIME"][:5]`
  - `data` は NumPy の構造化配列で、`["カラム名"]` で特定のカラムを取り出せます。
  - さらに `[:5]` で先頭 5 要素だけを確認しています。


## 4. `BaseFlareDetector.load_TESS_data` の挙動を軽く追う

ここまでで「生の FITS ファイル」の構造が分かったので、
`BaseFlareDetector` がどのようにそれを読み込んでいるかを、
EK Dra 用のクラス `FlareDetector_EK_Dra` を通して確認します。

- ファイル名から `data_name` やセクタ番号を抽出
- セクタ閾値 `sector_threshold` に応じて `PDCSAP_FLUX` か `SAP_FLUX` を選択
- NaN を除去し、`flux_mean` で割って正規化した配列を作る

ここでは、実装詳細を見るのではなく、
**"どんな配列が用意されるのか"** に注目して確認します。


In [ ]:
from src.flarepy_EK_Dra import FlareDetector_EK_Dra

if not DATA_DIR.exists() or not fits_files:
    print("EK Dra の FITS ファイルがないため、このセルはスキップします。")
else:
    example_file = fits_files[0]
    print("example_file:", example_file)

    # process_data=False とすることで、前処理だけを行った状態のインスタンスを得る
    det = FlareDetector_EK_Dra(file=str(example_file), process_data=False)

    print("data_name:", det.data_name)
    print("len(atessBJD):", len(det.atessBJD))
    print("len(amPDCSAPflux):", len(det.amPDCSAPflux))
    print("flux_mean (正規化の基準):", det.flux_mean)

    # 先頭数点だけ確認
    print("first 5 atessBJD:", det.atessBJD[:5])
    print("first 5 amPDCSAPflux:", det.amPDCSAPflux[:5])

### NaN 除去とブールマスクのミニ例

`load_TESS_data()` では、`PDCSAP_FLUX`（や `SAP_FLUX`）の NaN を除外するために、
NumPy のブールマスクが使われています。

ここでは、**小さな配列でブールマスクの挙動を確認**し、そのあと
実データにどのように適用されているかをイメージできるようにします。


In [ ]:
# 小さな例でブールマスクの基本を確認
arr = np.array([1.0, np.nan, 3.0, np.nan, 5.0])
print("arr:", arr)

mask = ~np.isnan(arr)
print("mask:", mask)

clean = arr[mask]
print("clean:", clean)

### ブールマスクの解説

- `np.isnan(arr)`
  - 各要素が NaN かどうかを判定し、`[False, True, False, True, False]` のような
    ブール配列を返します。
- `~np.isnan(arr)`
  - `~` はブール値に対する否定（NOT）で、`True` ↔ `False` を反転します。
  - "NaN ではない場所" を `True` にしたマスクになります。
- `arr[mask]`
  - ブール配列を添字として渡すと、`True` の要素だけを取り出した新しい配列になります。
  - 実データでは、`time` / `flux` / `flux_err` など **複数の配列に同じマスク** を適用することで、
    NaN を含む時刻を一括で取り除いています。

`BaseFlareDetector.load_TESS_data()` では、この仕組みを用いて
NaN を含まない時刻・フラックス・誤差の配列を揃え、その後の解析の土台を作っています。
